<a href="https://colab.research.google.com/github/ShantBalolian/Capstone-SoundClassification/blob/master/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA PREPROCESSING


In [ ]:
# DATA PREPROCESSING
# MFCCs
import pandas as pd
import numpy as np
import tables  
import librosa
import librosa.display

# time taken to read data
chunk = pd.read_csv('/content/esc50.csv',usecols=['filename','category'], chunksize=1000)
df = pd.concat(chunk)  
# data
#display(df)
classes = df['category']
audiofiles = df['filename']

#display(audioFiles)
mfccFeatures = []
i = 0
for wavFile in audiofiles:
    data, samplingRate = librosa.load(('/content/' + wavFile), sr=22050)

    #Grab the mfcc features through the mfcc algorithm
    mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
    mfccs_scaled_features = np.mean(mfcc.T,axis=0)


    mfccFeatures.append([mfccs_scaled_features,classes[i]])
    i += 1
    if (i == 1250):
        break
pd.set_option('display.max_colwidth', None)
mfccFeatures = pd.DataFrame(mfccFeatures, columns=['feature','class_category'])
# mfccFeaturesTable.to_excel("extractedMFCCFeaturesDataSet3.xlsx")
# mfccFeaturesTable.head()
print(type(mfccFeatures))



<class 'pandas.core.frame.DataFrame'>


# MODEL CREATION AND TESTING

In [ ]:
# MODEL CREATION AND TESTING

import pandas as pd
import numpy as np
import tables  
import librosa
import librosa.display
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier

# dataset = pd.read_csv('/content/extractedMFCCFeaturesDataSet1.csv')
X = np.array(mfccFeatures['feature'].tolist())
Y = np.array(mfccFeatures['class_category'].tolist())

#Encode target labels with value between 0 and n-1 classes
Label_encoder = LabelEncoder()
yy=to_categorical((Label_encoder.fit_transform(Y)))

print(yy.shape)
print(X.shape)

# clf = DecisionTreeClassifier(random_state=42)
# ss = ShuffleSplit(train_size=0.6, test_size=0.3, n_splits = 5)
# scores = cross_val_score(clf, X, yy, cv = ss)

# print("Cross Validation Scores: ", scores)
# print("Average CV Score: ", scores.mean())
# print("Number of CV Scores used in Average: ", len(scores))

#performing train test split on our data set.
X_train,X_test,Y_train,Y_test=train_test_split(X,yy,test_size=0.2,random_state=0)

num_labels=yy.shape[1]

#The first layer has 256 neurons. Input shape is 40 according to the number of features with activation
#function as Relu, and to avoid any overfitting, we’ll use the Dropout layer at a rate of 0.5.
#The second layer has 200 neurons with activation function as Relu and the drop out at a rate of 0.5.
#The third layer again has 100 neurons with activation as Relu and the drop out at a rate of 0.5.
#Performing softmax regression

model=Sequential()
###first layer
model.add(Dense(256,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# # ###third layer
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

# To compile the model we need to define loss function which is categorical cross-entropy,
# accuracy metrics which is accuracy score, and an optimizer which is Adam.

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(X_test, Y_test, verbose=0)
accuracy = 100*score[1]

# TESTING BEFORE TRAINING
print(f'the accuracy {accuracy} before training')

# TRAINING
num_epochs = 300
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, Y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, Y_test), callbacks=[checkpointer] ,verbose=1)

# EVALUATING AFTER TRAINING
score = model.evaluate(X_train, Y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(X_test, Y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

(1250, 50)
(1250, 40)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 256)               10496     
                                                                 
 activation_17 (Activation)  (None, 256)               0         
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_18 (Dense)            (None, 50)                12850     
                                                                 
 activation_18 (Activation)  (None, 50)                0         
                                                                 
Total params: 23,346
Trainable params: 23,346
Non-trainable params: 0
_________________________________________________________________
the accuracy 2.8000000864267

NotFittedError: ignored

#PREDICTION

---



---



In [ ]:
filename="shantshant.wav"

#preprocess the audio file
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

#Reshape MFCC feature to 2-D array
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

x_predict=model.predict(mfccs_scaled_features) 
predicted_label=np.argmax(x_predict,axis=1)

prediction_class = Label_encoder.inverse_transform(predicted_label) 
print(prediction_class)

1/1 [==============================] - 0s 36ms/step
['cat']
